In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.0 MB/s eta 0:00:00


In [2]:
!pip install transformers

In [4]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import torch
from transformers import T5Tokenizer, T5Model
from scipy.stats import uniform

# Mount Google Drive
drive.mount('/content/drive')

# Load the T5 model and tokenizer from Hugging Face
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")

# Function to convert text to T5 embeddings
def convert_to_t5_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to get T5 embeddings for a text
    def get_t5_embeddings(text):
        inputs = tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        # Provide decoder input for T5
        # Here, we use the same input for the decoder as the encoder
        # You might need to adjust this based on your specific task
        decoder_input_ids = inputs.input_ids

        outputs = model(
            **inputs,
            decoder_input_ids=decoder_input_ids
        )
        return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

    # Generate embeddings for the text column
    embeddings = df[text_column].apply(lambda x: get_t5_embeddings(str(x)))
    # Stack embeddings into a 2D array
    embeddings = np.stack(embeddings.values)

    # Print the dimensions of the embeddings
    print(f"T5 embeddings dimensions for '{text_column}': {embeddings.shape}")

    # Return the embeddings and the original DataFrame
    return embeddings, df
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, flattening test_pred first
    # if it is a NumPy array
    test_pred_flat = test_pred.flatten() if isinstance(test_pred, np.ndarray) else test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred_flat)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred_flat, columns=all_labels).reindex(columns=all_labels, fill_value=0) # using flattened pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")


def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random Forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Navie Bayes")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Mlp")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG boost")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Adaboost")


def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Define RandomizedSearchCV for hyperparameter tuning for RandomForest
def random_forest_tune(X_train, y_train):
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': uniform(0.01, 0.1),
        'min_samples_leaf': uniform(0.01, 0.1)
    }
    rf = RandomForestClassifier()
    random_search = RandomizedSearchCV(rf, param_dist, n_iter=30, cv=5, verbose=1, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best RandomForest Parameters: {random_search.best_params_}")
    return random_search.best_estimator_

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for T5 embeddings
text_column = 'answer'

# Convert text column to T5 embeddings and get the original DataFrame
embeddings, original_data = convert_to_t5_embeddings(input_file, text_column)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42)

# Run hyperparameter tuning for Random Forest with RandomizedSearchCV
best_rf_model = random_forest_tune(X_train, y_train)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
T5 embeddings dimensions for 'answer': (681, 512)
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best RandomForest Parameters: {'max_depth': 20, 'min_samples_leaf': 0.010440224086691832, 'min_samples_split': 0.0596162896377139, 'n_estimators': 300}
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.7206
Train Precision: 0.7014
Train Recall: 0.7206
Train F1 Score: 0.6988
Train Confusion Matrix:
[[  8  10  14]
 [  2  56  74]
 [  3  30 279]]

Test Accuracy: 0.6634
Test Precision: 0.6437
Test Recall: 0.6634
Test F1 Score: 0.6531
Test Confusion Matrix:
[[  0   2  11]
 [  1  17  23]
 [  3  29 119]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      72.06%
Test Accuracy       66.34%
Train-Test Gap      5.72%
ROC-AUC             0.58
Overfitting         No


Running Random Forest model.

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7073
Test Precision: 0.7263
Test Recall: 0.7073
Test F1 Score: 0.7141
Test Confusion Matrix:
[[  2   6   5]
 [  4  22  15]
 [  4  26 121]]


Performance Table for Mlp:
Metric              Value     
------------------------------
Train Accuracy      99.37%
Test Accuracy       70.73%
Train-Test Gap      28.64%
ROC-AUC             0.70
Overfitting         Yes


Running XGBoost model...
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.7035
Test Recall: 0.7317
Test F1 Score: 0.7113
Test Confusion Matrix:
[[  1   3   9]
 [  1  17  23]
 [  1  18 132]]


Performance Table for XG boost:
Metric              Value     
---------

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model Metrics:
Train Accuracy: 0.7983
Train Precision: 0.7981
Train Recall: 0.7983
Train F1 Score: 0.7769
Train Confusion Matrix:
[[ 12   9  11]
 [  2  64  66]
 [  2   6 304]]

Test Accuracy: 0.7366
Test Precision: 0.7056
Test Recall: 0.7366
Test F1 Score: 0.7172
Test Confusion Matrix:
[[  1   6   6]
 [  2  16  23]
 [  1  16 134]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      79.83%
Test Accuracy       73.66%
Train-Test Gap      6.17%
ROC-AUC             0.66
Overfitting         No


Running SVM model...
SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]


Performance Table for SVM:
Metric              Val

In [6]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.decomposition import PCA
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including T5 embeddings and numeric features
def process_features(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Initialize T5 model and tokenizer
    model_name = "t5-small"  # Change to a larger T5 model if required
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5EncoderModel.from_pretrained(model_name)

    # Generate T5 embeddings for the text column
    def encode_text(texts):
        # Ensure 'texts' is a string or a list of strings
        if not isinstance(texts, (str, list)):
            # If not a string or list, convert it to a string
            texts = str(texts)
        if isinstance(texts, str):
            texts = [texts]  # If it's a single string, wrap it in a list
        elif not all(isinstance(item, str) for item in texts):
            # If any item in the list is not a string, convert it to a string
            texts = [str(item) for item in texts]
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    embeddings = np.vstack([encode_text(text) for text in df[text_column].tolist()])

    # Convert embeddings to DataFrame
    # The indentation of this line was corrected
    t5_df = pd.DataFrame(embeddings, columns=[f't5_{i}' for i in range(embeddings.shape[1])])

    # Combine numeric features and embeddings
    combined_df = pd.concat([df[numeric_features], t5_df], axis=1)

    # Ensure all column names are strings in the combined DataFrame
    combined_df.columns = combined_df.columns.astype(str)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, flattening test_pred first
    # if it is a NumPy array
    test_pred_flat = test_pred.flatten() if isinstance(test_pred, np.ndarray) else test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred_flat)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred_flat, columns=all_labels).reindex(columns=all_labels, fill_value=0) # using flattened pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_
# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Navie Bayes")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random FOrest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XGboost")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Adaboost")


def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for model_name, model_function in models.items():
        print(f"Running {model_name}...")
        model_function(X_train, y_train, X_test, y_test)
input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'

# Define numeric_features - Replace [] with your actual numeric features if any
numeric_features = []  # Or list the actual numeric features from your dataset like: ['age', 'income']

# Process features
X, df = process_features(input_file, text_column, numeric_features)

# Assuming target column is labeled 'confidence'
y = df['confidence'].apply(lambda x: x + 1)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
X_train_reduced, pca = apply_pca(X_train)
X_test_reduced = pca.transform(X_test)

# Run models
run_all_models(X_train_reduced, y_train, X_test_reduced, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 152
Running KNN...
Best Params: {'weights': 'distance', 'n_neighbors': 10}
KNN Model Metrics:
Train Accuracy: 1.0000
Train Precision: 1.0000
Train Recall: 1.0000
Train F1 Score: 1.0000
Train Confusion Matrix:
[[ 32   0   0]
 [  0 146   0]
 [  0   0 366]]

Test Accuracy: 0.7299
Test Precision: 0.6636
Test Recall: 0.7299
Test F1 Score: 0.6794
Test Confusion Matrix:
[[ 1  6  6]
 [ 2  6 19]
 [ 0  4 93]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      100.00%
Test Accuracy       72.99%
Train-Test Gap      27.01%
ROC-AUC             0.63
Overfitting         Yes


Running Random Forest...
Best Params: {'n_estimators': 50, 'min_samples_split': 5, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9945
Train Precision: 0.9945
Train Recall: 0.9945
Train F1 Score

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9890
Train Precision: 0.9890
Train Recall: 0.9890
Train F1 Score: 0.9889
Train Confusion Matrix:
[[ 31   1   0]
 [  0 142   4]
 [  0   1 365]]

Test Accuracy: 0.7153
Test Precision: 0.7323
Test Recall: 0.7153
Test F1 Score: 0.7068
Test Confusion Matrix:
[[ 2  7  4]
 [ 1 14 12]
 [ 0 15 82]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      98.90%
Test Accuracy       71.53%
Train-Test Gap      27.36%
ROC-AUC             0.70
Overfitting         Yes


Running XGBoost...
Best Params: {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 1.0000
Train Precision: 1.0000
Train Recall: 1.0000
Train F1 Score: 1.0000
Train Confusion Matrix:
[[ 32   0   0]
 [  0 146   0]
 [  0   0 366]]

Test Accuracy: 0.7591
Test Precision: 0.7242
Test Recall: 0.7591
Test F1 Score: 0.7098


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.8290
Train Precision: 0.8294
Train Recall: 0.8290
Train F1 Score: 0.8292
Train Confusion Matrix:
[[ 32   0   0]
 [  0 100  46]
 [  0  47 319]]

Test Accuracy: 0.6131
Test Precision: 0.6327
Test Recall: 0.6131
Test F1 Score: 0.6127
Test Confusion Matrix:
[[ 0  8  5]
 [ 1 15 11]
 [ 0 28 69]]


Performance Table for Adaboost:
Metric              Value     
------------------------------
Train Accuracy      82.90%
Test Accuracy       61.31%
Train-Test Gap      21.59%
ROC-AUC             0.63
Overfitting         Yes


Running Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8695
Train Precision: 0.8681
Train Recall: 0.8695
Train F1 Score: 0.8649
Train Confusion Matrix:
[[ 24   5   3]
 [  1  99  46]
 [  0  16 350]]

Test Accuracy: 0.7591
Test Precision: 0.7440
Test Recall: 0.7591
Test F1 Score: 0.7420
Test Confusion Matrix:
[[ 2  5  6]
 [ 2 15 10]
 [ 0 10 87]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      86.95%
Test Accuracy       75.91%
Train-Test Gap      11.04%
ROC-AUC             0.72
Overfitting         Yes


Running SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9908
Train Precision: 0.9909
Train Recall: 0.9908
Train F1 Score: 0.9908
Train Confusion Matrix:
[[ 32   0   0]
 [  0 141   5]
 [  0   0 366]]

Test Accuracy: 0.7591
Test Precision: 0.7695
Test Recall: 0.7591
Test F1 Score: 0.7543
Test Confusion Matrix:
[[ 3  7  3]
 [ 1 17  9]
 [ 1 12 84]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      99.08%
Test Accuracy       75.91%
Train-Test Gap      23.17%
ROC-AUC             0.76
Overfitting         Yes


